In [48]:
import numpy as np
import awkward as ak
import uproot

import hist
import topcoffea.modules.utils as utils

import mplhep as hep
import matplotlib.pyplot as plt

In [49]:
hep.style.use("CMS")
params = {'axes.labelsize': 20,
          'axes.titlesize': 20,
          'legend.fontsize':20, 
          'lines.markersize':5}
plt.rcParams.update(params)

In [50]:
def get_ratio_uncertainty(num_hist, denom_hist):
    xvals = num_hist.axes.centers[0]
    yvals_num = num_hist.values()
    yvals_denom = denom_hist.values()
    sigma_num = np.sqrt(num_hist.variances())
    sigma_denom = np.sqrt(denom_hist.variances())

    ratio = np.divide(yvals_num, yvals_denom)

    # calculation for error propagation for ratio = yavls_num/yvals_denom
    # generally, z=x/y; sigmaz = abs(z)sqrt((sigmax/x)^2+(sigmay/y)^2)
    sigma_y = np.multiply(np.abs(ratio), np.sqrt(np.add(np.square(np.divide(sigma_num, yvals_num)), np.square(np.divide(sigma_denom, yvals_denom)))))

    return sigma_y

In [51]:
def get_ratio_points(hist1, histSM):
    num = hist1.values()
    centers = hist1.axes.centers[0]
    denomSM = histSM.values()
    ratio = np.divide(num, denomSM)

    return centers, ratio

In [56]:
def make_plot_with_ratio(h_num, h_denom, label_num, label_denom, xlabel, title):
    ## get ratios and uncertainties
    centers, ratio = get_ratio_points(h_num, h_denom)
    uncert = get_ratio_uncertainty(h_num, h_denom)

    ## Initialize figure and axes
    # fig, ax = plt.subplots()
    fig, (ax, rax) = plt.subplots(
        nrows=2,
        ncols=1,
        figsize=(10,12),
        gridspec_kw={"height_ratios": (3, 1)},
        sharex=True
    )
    fig.subplots_adjust(hspace=.1)

    ## Plot histograms and ratio plot
    hep.histplot(h_num, ax=ax, stack=False, yerr=True, label=label_num)
    hep.histplot(h_denom, ax=ax, stack=False, yerr=True, label=label_num)
    rax.scatter(centers, ratio)
    rax.errorbar(centers, ratio, xerr = None, yerr = uncert, capsize=5, ls='none')

    ## Formatting
    ax.legend(loc = 'upper right', fontsize = 'small')
    ax.set_ylabel("Events", fontsize='medium')
    ax.set_xlabel("")
    rax.set_ylabel("Ratio", fontsize='medium')
    rax.set_xlabel(xlabel, fontsize="medium")
    rax.set_ylim([0, 2])
    rax.grid(axis='y')
    plt.figtext(0.13, 0.9, title, fontsize='small')

    return fig

In [57]:
fin1 = "2810_ttbarEFT_0_700.pkl.gz"
fin2 = "2810_ttbarEFT_700_900.pkl.gz"
fin3 = "2810_ttbarEFT_900_Inf.pkl.gz"
fin4 = "2810_ttbarEFT_nanogen.pkl.gz"
hists1 = utils.get_hist_from_pkl(fin1, allow_empty=False)
hists2 = utils.get_hist_from_pkl(fin2, allow_empty=False)
hists3 = utils.get_hist_from_pkl(fin3, allow_empty=False)
hists4 = utils.get_hist_from_pkl(fin4, allow_empty=False)
print(hists1)

{'njets': Hist(Regular(10, 0, 10, name='njets'), storage=Weight()) # Sum: WeightedSum(value=13.7115, variance=0.00960224), 'nleps': Hist(Regular(10, 0, 10, name='nleps'), storage=Weight()) # Sum: WeightedSum(value=13.7115, variance=0.00960224), 'ntops': Hist(Regular(10, 0, 10, name='ntops'), storage=Weight()) # Sum: WeightedSum(value=13.7115, variance=0.00960224), 'mtt': Hist(Regular(30, 0, 1500, name='mtt', label='GEN invariant mass of tops [GeV]'), storage=Weight()) # Sum: WeightedSum(value=13.7115, variance=0.00960224), 'lhe_mtt': Hist(Regular(30, 0, 1500, name='lhe_mtt', label='LHE invariant mass of tops [GeV]'), storage=Weight()) # Sum: WeightedSum(value=13.7115, variance=0.00960224), 'mll': Hist(Regular(16, 0, 800, name='mll', label='invariant mass of leptons [GeV]'), storage=Weight()) # Sum: WeightedSum(value=13.7115, variance=0.00960224), 'dr_leps': Hist(Regular(24, 0, 6, name='$\\Delta R$ (leading lepton, subleading lepton)'), storage=Weight()) # Sum: WeightedSum(value=13.7115

In [58]:
## add together central sample mass ranges for total central sample
for name in hists1:
    h_central = hists1[name]+hists2[name]+hists3[name]
    h_nanogen = hists4[name]
    fig = make_plot_with_ratio(h_central, h_nanogen, "central", "nanogen", name, "Reweighted to starting point")
    figname = "2810_"+name+"_stpt.png"
    fig.savefig(figname)
    plt.close(fig)

/tmp/ipykernel_26038/522512218.py:5: RuntimeWarning: invalid value encountered in divide
  ratio = np.divide(num, denomSM)
/tmp/ipykernel_26038/3457844619.py:8: RuntimeWarning: invalid value encountered in divide
  ratio = np.divide(yvals_num, yvals_denom)
/tmp/ipykernel_26038/3457844619.py:12: RuntimeWarning: invalid value encountered in divide
  sigma_y = np.multiply(np.abs(ratio), np.sqrt(np.add(np.square(np.divide(sigma_num, yvals_num)), np.square(np.divide(sigma_denom, yvals_denom)))))
/tmp/ipykernel_26038/522512218.py:5: RuntimeWarning: invalid value encountered in divide
  ratio = np.divide(num, denomSM)
/tmp/ipykernel_26038/3457844619.py:8: RuntimeWarning: invalid value encountered in divide
  ratio = np.divide(yvals_num, yvals_denom)
/tmp/ipykernel_26038/3457844619.py:12: RuntimeWarning: invalid value encountered in divide
  sigma_y = np.multiply(np.abs(ratio), np.sqrt(np.add(np.square(np.divide(sigma_num, yvals_num)), np.square(np.divide(sigma_denom, yvals_denom)))))
/tmp/ipy

In [59]:
fin1 = "2810_ttbarEFT_0_700_SM.pkl.gz"
fin2 = "2810_ttbarEFT_700_900_SM.pkl.gz"
fin3 = "2810_ttbarEFT_900_Inf_SM.pkl.gz"
fin4 = "2810_ttbarEFT_nanogen_SM.pkl.gz"
hists1 = utils.get_hist_from_pkl(fin1, allow_empty=False)
hists2 = utils.get_hist_from_pkl(fin2, allow_empty=False)
hists3 = utils.get_hist_from_pkl(fin3, allow_empty=False)
hists4 = utils.get_hist_from_pkl(fin4, allow_empty=False)
print(hists1)

{'njets': Hist(Regular(10, 0, 10, name='njets'), storage=Weight()) # Sum: WeightedSum(value=10.2417, variance=0.00582706), 'nleps': Hist(Regular(10, 0, 10, name='nleps'), storage=Weight()) # Sum: WeightedSum(value=10.2417, variance=0.00582706), 'ntops': Hist(Regular(10, 0, 10, name='ntops'), storage=Weight()) # Sum: WeightedSum(value=10.2417, variance=0.00582706), 'mtt': Hist(Regular(30, 0, 1500, name='mtt', label='GEN invariant mass of tops [GeV]'), storage=Weight()) # Sum: WeightedSum(value=10.2417, variance=0.00582706), 'lhe_mtt': Hist(Regular(30, 0, 1500, name='lhe_mtt', label='LHE invariant mass of tops [GeV]'), storage=Weight()) # Sum: WeightedSum(value=10.2417, variance=0.00582706), 'mll': Hist(Regular(16, 0, 800, name='mll', label='invariant mass of leptons [GeV]'), storage=Weight()) # Sum: WeightedSum(value=10.2417, variance=0.00582706), 'dr_leps': Hist(Regular(24, 0, 6, name='$\\Delta R$ (leading lepton, subleading lepton)'), storage=Weight()) # Sum: WeightedSum(value=10.2417

In [60]:
## add together central sample mass ranges for total central sample
for name in hists1:
    h_central = hists1[name]+hists2[name]+hists3[name]
    h_nanogen = hists4[name]
    fig = make_plot_with_ratio(h_central, h_nanogen, "central", "nanogen", name, "Reweighted to SM")
    figname = "2810_"+name+"_SM.png"
    fig.savefig(figname)
    plt.close(fig)

/tmp/ipykernel_26038/522512218.py:5: RuntimeWarning: invalid value encountered in divide
  ratio = np.divide(num, denomSM)
/tmp/ipykernel_26038/3457844619.py:8: RuntimeWarning: invalid value encountered in divide
  ratio = np.divide(yvals_num, yvals_denom)
/tmp/ipykernel_26038/3457844619.py:12: RuntimeWarning: invalid value encountered in divide
  sigma_y = np.multiply(np.abs(ratio), np.sqrt(np.add(np.square(np.divide(sigma_num, yvals_num)), np.square(np.divide(sigma_denom, yvals_denom)))))
/tmp/ipykernel_26038/522512218.py:5: RuntimeWarning: invalid value encountered in divide
  ratio = np.divide(num, denomSM)
/tmp/ipykernel_26038/3457844619.py:8: RuntimeWarning: invalid value encountered in divide
  ratio = np.divide(yvals_num, yvals_denom)
/tmp/ipykernel_26038/3457844619.py:12: RuntimeWarning: invalid value encountered in divide
  sigma_y = np.multiply(np.abs(ratio), np.sqrt(np.add(np.square(np.divide(sigma_num, yvals_num)), np.square(np.divide(sigma_denom, yvals_denom)))))
/tmp/ipy